In [1]:
import bibliograph as bg

bibtex_fname = "bibliograph/test_data/bibtex_test_data_short.bib"
entry_syntax_fname = "bibliograph/resources/default_bibtex_syntax.csv"

tn = bg.slurp_bibtex(
    bibtex_fname,
    entry_syntax_fname,
    syntax_case_sensitive=False,
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)

tn.strings

,string,node_type_id,date_inserted,date_modified
0,638,4,2022-07-19 13:17:27,<NA>
1,641,4,2022-07-19 13:17:27,<NA>
2,311,4,2022-07-19 13:17:27,<NA>
3,323,4,2022-07-19 13:17:27,<NA>
4,261,4,2022-07-19 13:17:27,<NA>
...,...,...,...,...
102,3,3,2022-07-19 13:17:27,<NA>
103,"@article{newkirk_daytime_1962,\n author = {New...",0,2022-07-19 13:17:27,<NA>
104,Shorthand.parse_items,2,2022-07-19 13:17:27,<NA>
105,"entry_node_type,item_label,item_node_type,item...",1,2022-07-19 13:17:27,<NA>


In [3]:
    import bibliograph as bg

    tn = bg.slurp_shorthand(
        'bibliograph/test_data/manual_annotation.shnd',
        "bibliograph/resources/default_entry_syntax.csv",
        link_syntax_fname="bibliograph/resources/default_link_syntax.csv",
        syntax_case_sensitive=False,
        item_separator='__',
        default_entry_prefix='wrk',
        space_char='|',
        na_string_values='!',
        na_node_type='missing',
        skiprows=2,
        comment_char='#'
    )

    shorthand_text_string = tn.strings.query('node_type_id == 0')
    assert (shorthand_text_string.string.squeeze().startswith(
        'This is stuff shorthand ignores'
    ))

    tn.node_metadata_tables

{'shorthand_text':    node_id  node_type_id space_char na_string_values na_node_type  \
 0       60             0          |                !      missing   
 
   item_separator  
 0             __  ,
 'shorthand_entry_syntax':    node_id  node_type_id  case_sensitive  allow_redundant_items
 0       62             1           False                  False}

In [ ]:
!pytest bibliograph/tests.py

In [ ]:
import pandas as pd
a = 'a'
pd._random_function(a)

In [ ]:
import bibliograph
tn = bibliograph.TextNet()

tn._insert_metadata_table_test


In [ ]:
import pandas as pd

df = pd.DataFrame()

df.wrong_name()